## Dependencies

In [1]:
!pip uninstall -y tensorflow numpy && pip install tensorflow-gpu==1.14.0 tf_sentencepiece

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
Uninstalling numpy-1.16.5:
  Successfully uninstalled numpy-1.16.5
     |████████████████████████████████| 377.0MB 63kB/s 
     |████████████████████████████████| 2.7MB 39.4MB/s 
     |████████████████████████████████| 20.4MB 40.1MB/s 
ERROR: stable-baselines 2.2.1 requires tensorflow>=1.5.0, which is not installed.
ERROR: magenta 0.3.19 requires tensorflow>=1.12.0, which is not installed.
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
!curl -o many_to_many_fine_tune.zip https://dl.dropboxusercontent.com/s/3ixjojbpho2emd8/many_to_many_fine_tune.zip?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0  42.5M      0  0:00:02  0:00:02 --:--:-- 42.4M


In [0]:
!unzip -q many_to_many_fine_tune.zip

In [4]:
!curl -o mlt.zip https://codeload.github.com/suyash/mlt/zip/master

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38721    0 38721    0     0   115k      0 --:--:-- --:--:-- --:--:--  115k


In [0]:
!unzip -q mlt.zip && mv mlt-master/mlt ./

## Many to Many Fine-Tuned Translation

This takes the pretrained model from Many-to-Many and trains on pt-fr and pt-en with pt used as the 4th factor that was in the model but was never used when training many-to-many

In [0]:
import numpy as np
import tensorflow as tf
import tf_sentencepiece as tfs

from mlt.evaluation import predict
from mlt.layers import Attention, ConditionalNormalization, Gelu, MultiplyConstant, PaddingAndLookaheadMask, PaddingMask, PositionalEncoding

In [7]:
tf.__version__

'1.14.0'

In [0]:
sess = tf.InteractiveSession()

In [0]:
encoding_model_file = "./final/sentencepiece/en_de_fr_pt_shared/encoding.model"

In [10]:
with sess.as_default():
    model = tf.keras.experimental.load_from_saved_model("final/model114", custom_objects={
        "MultiplyConstant": MultiplyConstant,
        "PositionalEncoding": PositionalEncoding,
        "PaddingMask": PaddingMask,
        "PaddingAndLookaheadMask": PaddingAndLookaheadMask,
        "Attention": Attention,
        "ConditionalNormalization": ConditionalNormalization,
        "Gelu": Gelu,
    })

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### English to Portuguese

__NOTE:__ This is Few-Shot.

In [0]:
a = tfs.encode(["This is a problem that we need to solve."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [12]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [15]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['é um problema que precisamos de resolver .',
  'isto é um problema que precisamos de resolver .',
  'este é um problema que precisamos de resolver .',
  'é um problema que temos de resolver .',
  'isto é um problema para resolver .'],
 array([0.48925328, 0.40772614, 0.33876425, 0.20045164, 0.15399095],
       dtype=float32))

### Portuguese to English

__NOTE__: This is few shot

In [0]:
a = tfs.encode(["Este é um problema que temos que resolver."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [20]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['it is a problem that we have to solve .',
  'is a problem that we have to solve .',
  'eastern is a problem that we have to solve .',
  'it is a problem that we need to solve .',
  'is a problem that we need to solve .'],
 array([0.13502961, 0.12296236, 0.1034094 , 0.10161183, 0.09352253],
       dtype=float32))

### Portuguese to French

__NOTE__: This is few shot

In [0]:
a = tfs.encode(["Este é um problema que temos que resolver."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [25]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(["c'est un problème que nous devons régler .",
  "c'est un problème auquel nous devons nous régler .",
  "eh bien , c'est un problème que nous devons régler .",
  "c'est un problème auquel nous devons résoudre .",
  "c'est un problème dont nous devons régler ."],
 array([0.29969594, 0.24713854, 0.23395146, 0.19806242, 0.19502234],
       dtype=float32))

### French to Portuguese

__NOTE__: This is few shot

In [0]:
a = tfs.encode(["c'est un problème que nous devons résoudre."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [30]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['é um problema que precisamos de resolver .',
  'é um problema que temos de resolver .',
  'é um problema que precisamos disso .',
  'é um problema que precisamos de solucionar .',
  'é um problema que temos que resolver .'],
 array([0.4475441 , 0.3857079 , 0.23617241, 0.20825288, 0.19200097],
       dtype=float32))

### Portuguese to German

__NOTE:__ This is zero-shot, training was done for en-fr, fr-de, fr-pt and pt-en

In [0]:
a = tfs.encode(["Este é um problema que temos que resolver."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [35]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['egonisch ist ein Problem, das wir begriffen müssen .',
  'eh , es ist ein Problem, das wir begriffen müssen .',
  'eh , es ist ein Problem, das wir bereichern müssen .',
  'eh , es ist ein Problem , das wir uns müssen .',
  'egonisch ist ein Problem, das wir begriffen müssen.'],
 array([0.11276496, 0.09135193, 0.07864954, 0.05628131, 0.04623901],
       dtype=float32))

### German to Portuguese

__NOTE:__ This is zero-shot, training was done for en-fr, fr-de, fr-pt and pt-en

In [0]:
a = tfs.encode(["Dies ist ein Problem, das wir lösen müssen."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 0.0, 1.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [40]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['isto é um problema que temos de resolver .',
  'isto é um problema que temos que resolver .',
  'isto é um problema que precisamos de resolver .',
  'é um problema que temos de resolver .',
  'é um problema que temos que resolver .'],
 array([0.50007445, 0.3520581 , 0.306182  , 0.24348892, 0.1991398 ],
       dtype=float32))

### English to French

In [0]:
a = tfs.encode(["This is a problem that we need to solve."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [45]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(["C'est un problème que nous devons résoudre.",
  "C'est un problème qui nous faut résoudre.",
  "C'est un problème qu'il faut résoudre.",
  "C'est un problème auquel nous devons résoudre.",
  "C'est un problème pour résoudre ce problème."],
 array([0.4547706 , 0.31329823, 0.25585213, 0.25111327, 0.21929555],
       dtype=float32))

### English to German

In [0]:
a = tfs.encode(["This is a problem that we need to solve."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [50]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Das ist ein Problem, das wir brauchen, um zu lösen.',
  'Dies ist ein Problem, das wir brauchen, um zu lösen.',
  'Das ist ein Problem, das wir beheben müssen.',
  'Es ist ein Problem, das wir beheben müssen.',
  'Das ist ein Problem, das wir brauchen.'],
 array([0.6969994 , 0.5009153 , 0.38670796, 0.2249565 , 0.22388114],
       dtype=float32))

### French to English

In [0]:
a = tfs.encode(["c'est un problème que nous devons résoudre."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [55]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['This is a problem that we need to resolve.',
  'This is a problem we need to resolve.',
  'This is a problem that we have to solve.',
  'This is a problem that we need to solve.',
  'This is a problem that we have to resolve.'],
 array([0.3886431 , 0.30511987, 0.30445454, 0.28317395, 0.2635785 ],
       dtype=float32))

### German to English

In [0]:
a = tfs.encode(["Hier finden Sie alle Antworten, die Sie brauchen."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([1.0, 0.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [60]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Here you will find all the answers you need.',
  'Here you can find all the answers you need.',
  "Here you'll find all the answers you need.",
  'Here you find all the answers you need.',
  'Here you will find all the answers, you need answers.'],
 array([0.52666706, 0.37181923, 0.25515792, 0.25378272, 0.20513035],
       dtype=float32))

### French to German

__NOTE:__ This is zero-shot, training was done for en-fr and en-de

In [0]:
a = tfs.encode(["c'est un problème que nous devons résoudre."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    tarf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [65]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Das ist ein Problem, das wir uns tun müssen.',
  'Das ist ein Problem, das wir uns ärkt wissen müssen.',
  'Das ist ein Problem, das wir uns ärkt.',
  'Das ist ein Problem, das wir ärkt.',
  'Das ist ein Problem, das wir uns ärkt haben.'],
 array([0.4368739 , 0.22346888, 0.18264575, 0.18028234, 0.16595228],
       dtype=float32))

### German to French

__NOTE:__ This is zero-shot, training was done for en-fr and en-de

In [0]:
a = tfs.encode(["Hier finden Sie alle Antworten, die Sie brauchen."], model_file=encoding_model_file, add_bos=True, add_eos=True)[0]

In [0]:
ids, probs = predict(
    model=model,
    inputs=a,
    inpf=tf.constant([0.0, 0.0, 1.0, 0.0]),
    tarf=tf.constant([0.0, 1.0, 0.0, 0.0]),
    bos_id=1,
    eos_id=2,
    beam_size=5,
    vocab_size=40960,
    alpha=1.0,
)

In [0]:
mask = tf.cast(tf.not_equal(ids, 0), tf.int32)
seq_len = tf.reduce_sum(mask, axis=-1)
probs = tf.math.exp(probs)

In [0]:
ids_, probs_, seq_len_ = sess.run([ids, probs, seq_len])

In [70]:
[x.decode("utf-8") for x in sess.run(tfs.decode(ids_[0], seq_len_[0], model_file=encoding_model_file))], probs_[0]

(['Vous trouverez ici toutes les réponses que vous avez besoin.',
  'Ici vous trouverez toutes les réponses que vous avez besoin.',
  'Ici vous trouverez toutes les réponses que vous devez.',
  'Vous trouverez ici toutes les réponses que vous devez.',
  'Ici, vous trouverez toutes les réponses que vous avez besoin.'],
 array([0.48981324, 0.38794842, 0.3431491 , 0.3424902 , 0.2872332 ],
       dtype=float32))

In [0]:
sess.close()